In [1]:
import socket
import sys
import traceback
from threading import Thread
import hashlib
import math

In [2]:
host="127.0.0.1"
port=8080 

soc=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
soc.bind((host, port))
soc.listen(5)

In [3]:
def modInverse(a,m) : 
    m0=m 
    y=0
    x=1
    if(m==1) : 
        return 0
    while(a>1) : 
        q=a//m 
        t=m 
        m=a%m 
        a=t 
        t=y 
        y=x-q*y 
        x=t 

    if(x<0): 
        x=x+m0 
  
    return x

In [4]:
def client_thread(connection, ip, port):
    is_active=True

    while is_active:
        client_input = connection.recv(4096)
        client_input = client_input.decode("utf8").rstrip()
        client_input = str(client_input)

        if ":q" in client_input:
            print("Client is requesting to quit")
            connection.close()
            print("Connection " + ip + ":" + port + " closed")
            is_active = False
        else:
            inputs=client_input.split()
            print(inputs)
            if int(inputs[0])==0:
                q= int(inputs[1])
                g= int(inputs[2])
                r1= int(inputs[3])
                r2= int(inputs[4])
                print('prime:',q)
                print('g:',g)
                print('r1:',r1)
                print('r2:',r2)
            elif int(inputs[0])==1:
                c=int(inputs[1])
                s=int(inputs[2])
                m=inputs[3]
                print('c',c)
                print('s',s)
                print('m',m)
                yA=modInverse(r1,q)
                A_d=(pow(g,s,q)*pow(yA,c,q))%q

                yB=modInverse(r2,q)
                B_d=(pow(r1,s,q)*pow(yB,c,q))%q

                hash=str(A_d)+str(B_d)+m
                hash_object = hashlib.sha1(hash.encode())
                hex_dig = hash_object.hexdigest()
                c_here=int(hex_dig, 16)
                print('hex_dig',hex_dig)
                print('c_here', c_here)
                if c_here==c:
                    print("Matched")
                    message="matched: "+m
                    connection.sendall(message.encode("utf8"))
                else:
                    print("Not matched")
                    message="not matched"
                    connection.sendall(message.encode("utf8"))

In [ ]:
while True:
    connection, address = soc.accept()
    ip, port = str(address[0]), str(address[1])
    print("Connected with " + ip + ":" + port)
    Thread(target=client_thread, args=(connection, ip, port)).start()

soc.close()

Connected with 127.0.0.1:7245
['0', '100003219', '43', '73763837', '599036']
prime: 100003219
g: 43
r1: 73763837
r2: 599036
['1', '279924126212164518290617316870181359229179095867', '7194357680267335288874466431994901261503695631678039828', 'HelloClient']
c 279924126212164518290617316870181359229179095867
s 7194357680267335288874466431994901261503695631678039828
m HelloClient
hex_dig 31083b608d9bf5c6b8a21de584e83c69144d6b3b
c_here 279924126212164518290617316870181359229179095867
Matched
Client is requesting to quit
Connection 127.0.0.1:7245 closed
